# This is not working good enough --> Made new one called WS-fake_api

In [1]:
import requests
import pandas as pd
import selenium.common.exceptions
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import datetime as dt
import time
import re
import os
import numpy as np
import pymongo
import json
from fake_useragent import UserAgent


# Settings
options = webdriver.ChromeOptions()
options.headless = True
ua = UserAgent()
userAgent = ua.chrome
options.add_argument(f"user-agent={userAgent}")
options.add_argument('--window-size=1920,1080')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
start_url = "https://www.coinwarz.com/charts/difficulty-charts"
wait = WebDriverWait(driver, 1)
time1 = dt.datetime.now()
current_time = time1.strftime("%Y-%m-%dT%H_%M_%S")
client = pymongo.MongoClient()

C:\Users\Alibaba\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\Alibaba\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\Alibaba\AppData\Local\Temp/ipykernel_9104/417498474.py:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [2]:
def element_by_id(id1: str):
    element = driver.find_element(by=By.ID, value=id1)
    return element


def element_by_selector(selector: str):
    element = driver.find_element(by=By.CSS_SELECTOR, value=selector)
    return element

In [3]:
def crypto_button_click(name):
    # Making a button path is #coin_name > div.coinCardIcon > a > img
    crypto_button = element_by_selector(f"#{name} > div.coinCardIcon > a > img")
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(crypto_button)).click()
    # crypto_button.click()
    time.sleep(5)

In [4]:
def scrape_coin(name):
    driver.get(start_url)
    time.sleep(10)
    crypto_button_click(name)
    table_data = driver.find_element(By.CLASS_NAME, "table-responsive")
    data = table_data.text.split("\n")
    return data

In [5]:
def clean_coin(coin_data:list) -> dict:
    coin_name = coin_data[0].split(" ")[1]
    all_data = []

    for index, cell in enumerate(coin_data[1:]):
        coin_data_dict = {"coin":coin_name}
        cell_list = cell.split(" ")
        coin_data_dict["date"] = cell_list[0]
        coin_data_dict["difficulty"] = cell_list[1].split(".")[0]
        # coin_data_dict[cell_list[0]] = cell_list[1].split(".")[0]
        all_data.append(coin_data_dict)

    return all_data

In [6]:
def write_mongodb(data):
    # Make df
    df = pd.DataFrame(data)
    coin_name = df.iloc[0, 0]

    # Connect db
    db = client["Coin-Data"]
    col = db[coin_name]
    col.insert_one({"date": current_time, "data":df.to_dict(orient="records")})


In [7]:
def get_all_crypto_names():
    coin_div = element_by_selector("body > main > div:nth-child(4) > div > section")
    # print(coin_div.text.split("Difficulty"))
    coins = driver.find_elements(By.CLASS_NAME, "diffCoinCard")
    # print(len(coins))
    coin_list = []
    for coin in coins:
        temp_str = coin.text.split(" ")[0].split(")")[0][1:].lower()
        coin_list.append(temp_str)
        # print(temp_str)
    coin_list = list(set(coin_list))
    # print(coin_list)
    return coin_list

In [8]:
def get_popular_crypto_names():
    # coins = WebDriverWait(driver, 3).until(EC.presence_of_element_located(element_by_selector("body > main > div:nth-child(4) > div > section")))
    coins = element_by_selector("body > main > div:nth-child(4) > div > section")
    data = coins.text.split("\n")

    coins_list = []
    for coin in data:
        if coin[0] == "(" and coin[-1] == ")":
            # print(coin)
            coins_list.append(coin[1:-1].lower())
    # print(coins_list)
    return coins_list

In [9]:
def skip_add():
    try:
        WebDriverWait(driver, 2).until(EC.element_to_be_clickable(element_by_selector('#CloseCoinzillaSticky'))).click()
    except:
        print("No add found")

In [10]:
def main():
    start_time = dt.datetime.now()
    driver.get(start_url)
    time.sleep(10)
    skip_add()

    # Get all cryptocoin names
    buttons = get_popular_crypto_names()

    # Loop over all coins and get history for them
    all_data = []
    for coin in buttons:
        data = scrape_coin(coin)
        all_data.append(data)

    # Clean the coin data
    result_list = []
    for data in all_data:
        skip_add()
        result = clean_coin(data)
        result_list.append(result)

    for index, coin_data in enumerate(result_list):
        write_mongodb(coin_data)

    print(f"Runtime: {dt.datetime.now() - start_time} seconds")
    driver.quit()

In [11]:
if __name__ == '__main__':
    main()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Alibaba\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Alibaba\AppData\Local\Temp/ipykernel_9104/217905245.py", line 2, in <module>
    main()
  File "C:\Users\Alibaba\AppData\Local\Temp/ipykernel_9104/2330097330.py", line 3, in main
    driver.get(start_url)
  File "C:\Users\Alibaba\AppData\Local\Programs\Python\Python39\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 447, in get
    self.execute(Command.GET, {'url': url})
  File "C:\Users\Alibaba\AppData\Local\Programs\Python\Python39\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 433, in execute
    response = self.command_executor.execute(driver_command, params)
  File "C:\Users\Alibaba\AppData\Local\Programs\Python\Python39\lib\site-packages\selenium\webdriver\remote\remote_connection.py", line 344, in execute


TypeError: object of type 'NoneType' has no len()